In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.tabular.all import *
from fastai.callback import *

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
PATH = '/kaggle/input/tabular-playground-series-jan-2021/'

In [ ]:
train = pd.read_csv(PATH+'/train.csv')
test = pd.read_csv(PATH+'test.csv')
submission = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
[i.shape for i in [train, test, submission]]

In [ ]:
train.describe()

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

cont_name = [col for col in train.columns if col != 'target']
target_col = ['target']

In [ ]:
procs = [FillMissing, Normalize]

data = TabularDataLoaders.from_df(train, 
                                  y_names=target_col,
                                  procs=procs, 
                                  cont_names=cont_name,
                                  bs=512,
                                  num_workers=8,
                                  shuffle_train=True,
                                  drop_last=True,
                                  device='cuda')

In [ ]:
def predict(learn, data):
    dl = learn.dls.test_dl(data)
    pred, _ = learn.get_preds(dl=dl)        
    pred_scored = pred.numpy()
    return pred_scored

In [ ]:
preds = []

for i in range(5):
    print('='*30+f' Round {i} '+'='*30)
    learn = tabular_learner(data,  
                            layers=[512, 256, 256, 128], 
                            y_range=(train['target'].min(), train['target'].max()),
                            loss_func = MSELossFlat(),
                            opt_func= SGD,
                           n_out=1)
    print(learn.model)
    name = 'best_model_' + str(i) 
    
    callbacks = [
    EarlyStoppingCallback(monitor='valid_loss', min_delta=1e-5, patience=10),    
    ReduceLROnPlateau(monitor='valid_loss', min_delta=0.1, patience=5, min_lr=1e-8),
    SaveModelCallback(monitor='valid_loss',fname=name)] 

    
    lr = 0.1e-1
    learn.fit_one_cycle(15, slice(lr/(2.6**4),lr), wd=0.01, cbs=callbacks, reset_opt= True)
    learn.load(name)
    
    preds.append(predict(learn, test))

preds = np.array(preds).mean(axis=0)

In [ ]:
pred = submission.copy(deep=True)
pred['target'] = preds
pred.to_csv('submission.csv', index = False)